# Building a RegEx engine by using Finite Automata based approach

# Finite Automata & Regular Expressions are just **Pattern Matching**      
Regular expressions *describe* patterns in text.  
Finite automata (DFAs and NFAs) are abstract machines that can *recognize* those patterns.       
A finite automaton is, at its core, a graph. It has nodes (**states**) and directed edges (**transitions**).      
Our goal is to write a Python program that can "walk" this graph based on an input string.

We go:
**Regular Expression** `(a|b)*c` → **Thompson's Algorithm** → **NFA** (Nondeterministic Finite Automaton) → **Subset Construction** → **DFA** (Deterministic Finite Automaton)

## DFA & NFA - [**Finite State Machines**](https://en.wikipedia.org/wiki/Finite-state_machine)           
           
[Finite automata](https://www.geeksforgeeks.org/theory-of-computation/introduction-of-finite-automata/) are simple "machines".             
They have a finite number of **states** and move between them based on input symbols.              
Kind of like a Monopoly or snakes & ladders or similar board game, where each space is a state and the input tells you which path to take.             
           
[Here's a course I've enjoyed](https://www.youtube.com/playlist?list=PL7HjUNIdk93ThXvz2Oa_g30Jt3Owwm4HZ), hopefully more folks do too.           
         

  
### [Deterministic Finite Automaton (DFA)](https://en.wikipedia.org/wiki/Deterministic_finite_automaton)           
           
A DFA is strict, predictable.   
For any given state and any given input symbol, there is **exactly one** state it can move to.    
No ambiguity.           

                      
           

1.  **One Path:** For any input string, there's only one possible path through the machine.           
2.  **No $\epsilon$-transitions:** The machine can't change state without consuming an input symbol (no "free" moves).           
3.  **Every state must have a transition for every symbol** in the alphabet.           
           

A DFA has 5 parts - States, Alphabet, Transitions, Start State, Accept States:            
           

  * $Q$: A finite set of *states*.           
  * $\Sigma$: A finite set of input *symbols* (aka the alphabet).           
  * $\delta$: The transition *function*, $\delta: Q \times \Sigma \to Q$. (Given a state and a symbol, it returns *one* next state).           
  * $q_0$: The initial start state.           
  * $F$: A set of *final* or "*accept*" states.           

*(This Greek thing is what's formally used, IMO its just dissonance, takes longer for our brains to grok the main idea)*

**Example: A DFA that accepts strings ending in 'a'**           
           
  * $Q$ - The set of **states:** {$State_0$, $State_1$}           
  * $q_0$ - **Start State:** $State_0$           
  * **Accept State:** {$State_1$}           
  * **Transitions:**           
      * From $State_0$, on 'a' go to $State_1$.           
      * From $State_0$, on 'b' go to $State_0$.           
      * From $State_1$, on 'a' go to $State_1$.           
      * From $State_1$, on 'b' go to $State_0$.          
           

tracing the execution of the state machine for the input string "**bab**a":           
           
1.  Start at $State_0$. Input 'b' -> Stay at $State_0$.           
2.  At $State_0$. Input 'a' -> Move to $State_1$.           
3.  At $State_1$. Input 'b' -> Move to $State_0$.           
4.  At $State_0$. Input 'a' -> Move to $State_1$.           
       
...end of string. We are in S1, which is an accept state. **The string is accepted!**            
          

#### The key rule for a DFA is: from any given state, for any given input symbol, there is **exactly one** place to go next.     
Another heuristic is to draw the machine as a graph, where as many arrows exit the node as enter.   
Also, every symbol in the alphabet is addressed in each node.

#### State Machine **Execution Engine**  

`Engine` is a big word for a simple function.   
It expects a dictionary of the following format and cycles through any input string to `execute`...   
```Python
dfa = {
    "states":{},         # dict of states - "state_name"
    "alphabet":{},       # dict of alphabets - "alphabet"
    "transitions":{},    # dict of transitions - {"state_name": {"input_alphabet":"next_state", "input_alphabet":"next_state"...}, ...}
    "start_state":"",    # string - the "state_name" of the start state
    "accept_states":{}   # dict of accept states
}
```

In [22]:
# RUN DA DFA - a method to run the DFA and provide a trace of how the machine processed things
# It expects a DFA with states, alphabet, transitions, start_state, accept_states
def run_da_dfa(machine, input_string):

    current_state = machine["start_state"]
    print(f"Start: state = {current_state}")

    # Process the string one symbol at a time.
    for symbol in input_string:
        # Check if the symbol is valid for this machine.
        if symbol not in machine["alphabet"]:
            print(f"  Error: Symbol '{symbol}' not in alphabet. Rejecting.")
            return False

        # Look up the next state from the transitions table.
        next_state = machine["transitions"][current_state][symbol]
        print(f"  Input '{symbol}': {current_state} -> {next_state}")
        current_state = next_state

    # After the loop, check if the final state is an accept state.
    is_accepted = current_state in machine["accept_states"]
    print(f"End: final state = {current_state}. Accepted: {is_accepted}\n")
    return is_accepted

#### Example 01:  **strings that contain an even number of 'a's**.   
The alphabet is {'a', 'b'}.

  * `"aba"` -\> accepted (2 'a's)
  * `"b"` -\> accepted (0 'a's)
  * `"a"` -\> rejected (1 'a')
  * `"aaab"` -\> rejected (3 'a's)

We build a machine with two states:
  
1.  `q0`: even number of 'a's seen in the string so far. This is our **start state**. Since 0 is even, it is also an **accept state**.
2.  `q1`: encountered an odd number of 'a's so far.
   

represent this entire machine as a Python dictionary.   
This is just a data structure - there's no complex logic in it yet.

```mermaid
stateDiagram-v2
    direction LR
    [*] --> q0
    q0 --> q1: a
    q0 --> q0: b
    q1 --> q0: a
    q1 --> q1: b
    q0: q0 (both, start state and accept state)
```

In [23]:
# DFA = {states, alphabets, transitions, start_state, accept_states}
even_a_machine = {
    # A set of all states for this machine.
    "states": {"q0", "q1"},
    # The set of allowed input symbols.
    "alphabet": {"a", "b"},
    # The transition function, mapping (state, symbol) to a next state.
    # For a DFA, the next state is a single state.
    "transitions": {
        "q0": {"a": "q1", "b": "q0"},
        "q1": {"a": "q0", "b": "q1"},
    },
    # The single state where the machine begins.
    "start_state": "q0",
    # A set of states that mean "accept" if the machine finishes in one of them.
    "accept_states": {"q0"},
}

In [24]:
print("Testing '':")
run_da_dfa(even_a_machine, "")     # Expected: True (0 is even)

Testing '':
Start: state = q0
End: final state = q0. Accepted: True



True

In [25]:
print("Testing 'aba':")
run_da_dfa(even_a_machine, "aba")  # Expected: True

Testing 'aba':
Start: state = q0
  Input 'a': q0 -> q1
  Input 'b': q1 -> q1
  Input 'a': q1 -> q0
End: final state = q0. Accepted: True



True

In [26]:
print("Testing 'a':")
run_da_dfa(even_a_machine, "a")    # Expected: False

Testing 'a':
Start: state = q0
  Input 'a': q0 -> q1
End: final state = q1. Accepted: False



False

In [27]:
print("Testing 'bbab':")
run_da_dfa(even_a_machine, "bbab") # Expected: False

Testing 'bbab':
Start: state = q0
  Input 'b': q0 -> q0
  Input 'b': q0 -> q0
  Input 'a': q0 -> q1
  Input 'b': q1 -> q1
End: final state = q1. Accepted: False



False

**Data and Logic**     
The `even_a_machine` dictionary is pure data.       
The `run_da_dfa` function is pure logic.      
This is a fundamental concept.        
The same `run_dfa` function could run *any* DFA we define, as long as it follows the same dictionary structure.                

**State**    
The `current_state` variable is the machine's entire memory. It's a very limited memory, which is why it's a "finite" automaton. It only knows which state it's in; it doesn't remember the path it took to get there.                

**Determinism**      
Look at the `transitions` dictionary.     
For every state (`q0`, `q1`) and every symbol (`a`, `b`), there is **one and only one** resulting state specified.     
This certainty is what makes it deterministic.                 

#### Example 02: **accepts strings ending in '1'**    

For the binary alphabet - "0" and "1", we need  
a machine that *accepts* binary strings like "001", "11", and "1" but *rejects* "10", "00", and ""   



  * `q0`: The initial state. We are in this state if we haven't seen a '1' yet, or the last symbol seen was a '0'. This is a **reject** state.                
  * `q1`: The state we are in if the most recent symbol was a '1'. This is an **accept** state.                

```mermaid
stateDiagram-v2
    direction LR
    [*] --> q0
    q0 --> q0: 0
    q0 --> q1: 1
    q1 --> q0: 0
    q1 --> q1: 1
    q1: q1 (accept state)
    q0: q0 (reject state)
```

In [18]:
dfa_ends_with_1 = {                
    "states": {"q0", "q1"},                
    "alphabet": {"0", "1"},                
    "transitions": {                
        # From state q0: if we read a '0', we stay in q0. If '1', we go to q1.                
        "q0": {"0": "q0", "1": "q1"},                
        # From state q1: if we read a '0', we go back to q0. If '1', we stay in q1.                
        "q1": {"0": "q0", "1": "q1"},                
    },                
    "start_state": "q0",                
    "accept_states": {"q1"},                
}                

In [28]:
print("Testing '101':")
run_da_dfa(dfa_ends_with_1, "101")                

Testing '101':
Start: state = q0
  Input '1': q0 -> q1
  Input '0': q1 -> q0
  Input '1': q0 -> q1
End: final state = q1. Accepted: True



True

In [29]:
print("Testing '100':")
run_da_dfa(dfa_ends_with_1, "100")

Testing '100':
Start: state = q0
  Input '1': q0 -> q1
  Input '0': q1 -> q0
  Input '0': q0 -> q0
End: final state = q0. Accepted: False



False

#### Example 03: strings that **start with "ab"**  
The alphabet is {'a', 'b'}.

```mermaid
stateDiagram-v2
    direction LR
    [*] --> q0 : Start
    q0 --> q1: a
    q0 --> q_dead: b
    q1 --> q2: b
    q1 --> q_dead: a
    q2 --> q2: a
    q2 --> q2: b
    q_dead --> q_dead: a
    q_dead --> q_dead: b
    q2: Accept
    q_dead: Reject
```

In [33]:
dfa_starts_with_ab = {                
    "states": {"q0", "q1", "q2", "q_dead"},                
    "alphabet": {"a", "b"},                
    "start_state": "q0",                
    "accept_states": {"q2"},                
    "transitions": {                
        "q0": {"a": "q1", "b": "q_dead"},         # Must start with 'a' - if 'a' is found at start, go to 'q1', where we check for 'b'                
        "q1": {"a": "q_dead", "b": "q2"},         # Then must have 'b' - if 'b' is also found, go to 'q2' which is our 'accept_state'...                
        "q2": {"a": "q2", "b": "q2"},             # It started with "ab", so accept                
        "q_dead": {"a": "q_dead", "b": "q_dead"}, # Any other path fails                
    },                
}

In [34]:
run_da_dfa(dfa_starts_with_ab, "abbab") # Should accept

Start: state = q0
  Input 'a': q0 -> q1
  Input 'b': q1 -> q2
  Input 'b': q2 -> q2
  Input 'a': q2 -> q2
  Input 'b': q2 -> q2
End: final state = q2. Accepted: True



True

In [35]:
run_da_dfa(dfa_starts_with_ab, "aab")   # Should reject

Start: state = q0
  Input 'a': q0 -> q1
  Input 'a': q1 -> q_dead
  Input 'b': q_dead -> q_dead
End: final state = q_dead. Accepted: False



False

#### Example 04: strings of **exactly length 2**     
    The alphabet is {'a', 'b'}.

```mermaid
stateDiagram-v2
    direction LR
    [*] --> q0: Start
    q0 --> q1: a
    q0 --> q1: b
    q1 --> q2: a
    q1 --> q2: b
    q2 --> q_dead: a
    q2 --> q_dead: b
    q_dead --> q_dead: a
    q_dead --> q_dead: b
    q2: Accept
    q_dead: Reject
```

In [37]:
dfa_len_2 = {                
    "states": {"q0", "q1", "q2", "q_dead"},                
    "alphabet": {"a", "b"},                
    "start_state": "q0",                
    "accept_states": {"q2"},                
    "transitions": {                
        "q0": {"a": "q1", "b": "q1"},           # First character                
        "q1": {"a": "q2", "b": "q2"},           # Second character                
        "q2": {"a": "q_dead", "b": "q_dead"},   # Third character (too long)                
        "q_dead": {"a": "q_dead", "b": "q_dead"},                
    },                
}

In [38]:
run_da_dfa(dfa_len_2, "ab") # Should accept

Start: state = q0
  Input 'a': q0 -> q1
  Input 'b': q1 -> q2
End: final state = q2. Accepted: True



True

In [39]:
run_da_dfa(dfa_len_2, "a")  # Should reject

Start: state = q0
  Input 'a': q0 -> q1
End: final state = q1. Accepted: False



False

#### Example 05: strings with an **odd number of '1's**  

The alphabet is {'1', '0'}.

```mermaid
stateDiagram-v2
    direction LR
    [*] --> q_even: Start
    q_even --> q_even: 0
    q_even --> q_odd: 1
    q_odd --> q_odd: 0
    q_odd --> q_even: 1
    q_odd: Accept
```

In [40]:
dfa_odd_ones = {                
    "states": {"q_even", "q_odd"},                
    "alphabet": {"0", "1"},                
    "start_state": "q_even",                
    "accept_states": {"q_odd"},                
    "transitions": {                
        "q_even": {"0": "q_even", "1": "q_odd"},                
        "q_odd": {"0": "q_odd", "1": "q_even"},                
    },                
}

In [41]:
run_da_dfa(dfa_odd_ones, "01101") # Should accept

Start: state = q_even
  Input '0': q_even -> q_even
  Input '1': q_even -> q_odd
  Input '1': q_odd -> q_even
  Input '0': q_even -> q_even
  Input '1': q_even -> q_odd
End: final state = q_odd. Accepted: True



True

In [42]:
run_da_dfa(dfa_odd_ones, "11")    # Should reject

Start: state = q_even
  Input '1': q_even -> q_odd
  Input '1': q_odd -> q_even
End: final state = q_even. Accepted: False



False

#### Example 06: accepting binary numbers that are a **multiple of 3**     
   
This is both a bad example and a clever one.    
Relies on how multiples of 3 look like in binary.

```mermaid
stateDiagram-v2
    direction LR
    [*] --> q0: Start
    q0 --> q0: 0
    q0 --> q1: 1
    q1 --> q2: 0
    q1 --> q0: 1
    q2 --> q1: 0
    q2 --> q2: 1
    q0: Accept
```

In [51]:
dfa_multiple_of_3 = {                
    "states": {"q0", "q1", "q2"},                
    "alphabet": {"0", "1"},                
    "start_state": "q0", # Remainder 0                
    "accept_states": {"q0"},                
    "transitions": {                
        # If current value is `x`, reading '0' makes it `2x`. Reading '1' makes it `2x+1`.                
        # All calculations are modulo 3.                
        "q0": {"0": "q0", "1": "q1"}, # (2*0)%3=0, (2*0+1)%3=1                
        "q1": {"0": "q2", "1": "q0"}, # (2*1)%3=2, (2*1+1)%3=0                
        "q2": {"0": "q1", "1": "q2"}, # (2*2)%3=1, (2*2+1)%3=2                
    },                
}

In [52]:
run_da_dfa(dfa_multiple_of_3, "110") # 6 in binary, should accept

Start: state = q0
  Input '1': q0 -> q1
  Input '1': q1 -> q0
  Input '0': q0 -> q0
End: final state = q0. Accepted: True



True

In [53]:
run_da_dfa(dfa_multiple_of_3, "101") # 5 in binary, should reject

Start: state = q0
  Input '1': q0 -> q1
  Input '0': q1 -> q2
  Input '1': q2 -> q2
End: final state = q2. Accepted: False



False

#### Example 07: strings with **no consecutive 'a's**    

```mermaid
stateDiagram-v2
    direction LR
    [*] --> q_start: Start
    q_start --> q_saw_a: a
    q_start --> q_start: b
    q_saw_a --> q_dead: a
    q_saw_a --> q_start: b
    q_dead --> q_dead: a
    q_dead --> q_dead: b
    q_start: Accept
    q_saw_a: Accept
    q_dead: Reject

```

In [46]:
dfa_no_consecutive_as = {                
    "states": {"q_start", "q_saw_a", "q_dead"},                
    "alphabet": {"a", "b"},                
    "start_state": "q_start",                
    "accept_states": {"q_start", "q_saw_a"},                
    "transitions": {                
        "q_start": {"a": "q_saw_a", "b": "q_start"},                
        "q_saw_a": {"a": "q_dead", "b": "q_start"},                
        "q_dead": {"a": "q_dead", "b": "q_dead"},                
    },                
}

In [47]:
run_da_dfa(dfa_no_consecutive_as, "ababa") # Should accept

Start: state = q_start
  Input 'a': q_start -> q_saw_a
  Input 'b': q_saw_a -> q_start
  Input 'a': q_start -> q_saw_a
  Input 'b': q_saw_a -> q_start
  Input 'a': q_start -> q_saw_a
End: final state = q_saw_a. Accepted: True



True

In [50]:
run_da_dfa(dfa_no_consecutive_as, "bababa") # Should accept

Start: state = q_start
  Input 'b': q_start -> q_start
  Input 'a': q_start -> q_saw_a
  Input 'b': q_saw_a -> q_start
  Input 'a': q_start -> q_saw_a
  Input 'b': q_saw_a -> q_start
  Input 'a': q_start -> q_saw_a
End: final state = q_saw_a. Accepted: True



True

In [48]:
run_da_dfa(dfa_no_consecutive_as, "aab")  # Should reject

Start: state = q_start
  Input 'a': q_start -> q_saw_a
  Input 'a': q_saw_a -> q_dead
  Input 'b': q_dead -> q_dead
End: final state = q_dead. Accepted: False



False

In [49]:
run_da_dfa(dfa_no_consecutive_as, "baab")  # Should reject

Start: state = q_start
  Input 'b': q_start -> q_start
  Input 'a': q_start -> q_saw_a
  Input 'a': q_saw_a -> q_dead
  Input 'b': q_dead -> q_dead
End: final state = q_dead. Accepted: False



False

       
### [Nondeterministic Finite Automaton (NFA)](https://en.wikipedia.org/wiki/Nondeterministic_finite_automaton)           
           
An NFA is flexible.    
It can have **multiple possible moves** from a single state on the same input symbol.  
           

1.  **Multiple Paths:** From a state, an input symbol can lead to zero, one, or multiple states.           
2.  **$\epsilon$-transitions (Epsilon-moves):** An NFA can change its state *without* consuming an input symbol. This is a "free" move.           
       

The NFA accepts an input string if **at least one** of the possible paths ends in an accept state.           
           

The formal definition is almost the same, but the transition function is different:           
           
  * $\delta$: The transition function, $\delta: Q \times (\Sigma \cup {\epsilon}) \to \mathcal{P}(Q)$.
      * (Given a state and a symbol (either symbol or $\epsilon$), it returns a *set* of possible next states).
      * $\mathcal{P}(Q)$ is the power set of $Q$.           
           

NFAs are often much simpler and smaller than their equivalent DFAs.    
I think of these as 'higher order' than DFAs, so you can write in NFA, let it compile to a DFA etc.  
  

NFAs easy to build from regular expressions, which is where Thompson's Algorithm comes in.           
 